In [1]:
from datasets import load_dataset
import grelu.resources
import grelu.variant
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import average_precision_score

/accounts/projects/yss/gbenegas/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/scratch/users/gbenegas/software/mambaforge/envs/gpn/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
dataset_path = "gonzalobenegas/siraj_gwas_highpip"
V = load_dataset(dataset_path, split="test").to_pandas()
V.chrom = "chr" + V.chrom
V

,chrom,pos,ref,alt,label
0,chr1,3080038,T,C,True
1,chr1,3774964,A,G,True
2,chr1,6616131,C,T,False
3,chr1,7665224,C,A,False
4,chr1,8407293,G,A,False
...,...,...,...,...,...
1778,chr22,47019717,G,T,False
1779,chr22,47990921,C,T,True
1780,chr22,50007172,T,C,False
1781,chr22,50190508,G,A,True


In [3]:
#precomputed = pd.read_parquet(f"https://huggingface.co/datasets/{dataset_path}/resolve/main/features/Enformer_absDelta.parquet")
#precomputed_cols = precomputed.columns
#V = pd.concat([V, precomputed], axis=1)
#V = V.dropna()
#precomputed = V[precomputed_cols].values
#V.shape

In [4]:
model = grelu.resources.load_model(project="enformer", model_name="human")
#model = grelu.resources.load_model(project="borzoi", model_name="human_fold0")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gonzalobenegas. Use `wandb login --relogin` to force relogin
wandb: Downloading large artifact human:latest, 940.48MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.7


In [23]:
L = model.data_params["train_seq_len"]
L

196608

In [26]:
V["start"] = V.pos - 1 - L // 2
V["end"] = V.start + L
V

,chrom,pos,ref,alt,label,start,end
0,chr1,3080038,T,C,True,2981733,3178341
1,chr1,3774964,A,G,True,3676659,3873267
2,chr1,6616131,C,T,False,6517826,6714434
3,chr1,7665224,C,A,False,7566919,7763527
4,chr1,8407293,G,A,False,8308988,8505596
...,...,...,...,...,...,...,...
1778,chr22,47019717,G,T,False,46921412,47118020
1779,chr22,47990921,C,T,True,47892616,48089224
1780,chr22,50007172,T,C,False,49908867,50105475
1781,chr22,50190508,G,A,True,50092203,50288811


In [31]:
import bioframe as bf


chromsizes = bf.fetch_chromsizes("hg38").to_frame().reset_index().rename(columns={"index": "chrom"})
chromsizes

,chrom,length
0,chr1,248956422
1,chr2,242193529
2,chr3,198295559
3,chr4,190214555
4,chr5,181538259
5,chr6,170805979
6,chr7,159345973
7,chr8,145138636
8,chr9,138394717
9,chr10,133797422


In [32]:
V = pd.merge(V, chromsizes, on="chrom", how="left")
V

,chrom,pos,ref,alt,label,start,end,length
0,chr1,3080038,T,C,True,2981733,3178341,248956422
1,chr1,3774964,A,G,True,3676659,3873267,248956422
2,chr1,6616131,C,T,False,6517826,6714434,248956422
3,chr1,7665224,C,A,False,7566919,7763527,248956422
4,chr1,8407293,G,A,False,8308988,8505596,248956422
...,...,...,...,...,...,...,...,...
1778,chr22,47019717,G,T,False,46921412,47118020,50818468
1779,chr22,47990921,C,T,True,47892616,48089224,50818468
1780,chr22,50007172,T,C,False,49908867,50105475,50818468
1781,chr22,50190508,G,A,True,50092203,50288811,50818468


In [33]:
V["within_bounds"] = (V.start >= 0) & (V.end <= V.length)
V["within_bounds"].value_counts()

within_bounds
True     1782
False       1
Name: count, dtype: int64

In [34]:
V[~V.within_bounds]

,chrom,pos,ref,alt,label,start,end,length,within_bounds
167,chr1,248897507,G,A,True,248799202,248995810,248956422,False


In [5]:
n = 4

In [19]:
delta = grelu.variant.predict_variant_effects(
    #variants=V.head(n),
    #variants=V,
    variants=V.iloc[167:170],
    model=model, 
    devices=0, # Run on GPU 0
    num_workers=8,
    batch_size=2,
    genome="hg38",
    compare_func="subtract",
    return_ad=False,
    rc = True, # Reverse complement the ref/alt predictions and average them.
)
delta.shape

making dataset


AssertionError: All input sequences must have the same length.

In [20]:
V.loc[167:170]

,chrom,pos,ref,alt,label
167,chr1,248897507,G,A,True
168,chr2,242793,G,C,True
169,chr2,417167,T,C,True
170,chr2,6586354,A,T,False


In [21]:
248956422 - 248897507

58915

In [8]:
sum_score = np.sum(delta, axis=2)
sum_score.shape

(4, 7611)

In [9]:
def log2fc_1p(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    return np.log2(1+x) - np.log2(1+y)

lfc = grelu.variant.predict_variant_effects(
    variants=V.head(n),
    model=model, 
    devices=0, # Run on GPU 0
    num_workers=8,
    batch_size=2,
    genome="hg38",
    compare_func=log2fc_1p,
    return_ad=False, # Return an anndata object.
    rc = True, # Reverse complement the ref/alt predictions and average them.
)
lfc.shape

making dataset


/scratch/users/gbenegas/software/mambaforge/envs/gpn/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/users/gbenegas/software/mambaforge/envs/gpn ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|█████████████████████████████████████| 8/8 [00:10<00:00,  0.74it/s]


(4, 7611, 6144)

In [10]:
l2_score = np.linalg.norm(lfc, axis=2)
l2_score.shape

(4, 7611)

## Benchmarking

In [13]:
V.head(n).label.mean()

0.5

In [14]:
average_precision_score(V.head(n).label, np.linalg.norm(precomputed[:n], axis=1))

1.0

In [15]:
average_precision_score(V.head(n).label, np.linalg.norm(sum_score[:n], axis=1))

1.0

In [16]:
average_precision_score(V.head(n).label, np.linalg.norm(l2_score[:n], axis=1))

1.0